In [33]:
# Initial imports
import os
import sys
import numpy as np

import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

# Import system files
from src import evaluate as test
from src import parsers as par
from src import data_loader as dl
from src import functions as f
from src.model import NN_Model

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

def main():
    # Test import
    test.test_print()
    
    # Import model
    # model = NN_Model().to(device)
    
    # load data e.g.:
    
    # training_data =
    # test_data = 
    
    # Create data loaders
    
    # Set hyperparameters for learning
    
    # loss_fn = nn.CrossEntropyLoss()
    # learning_rate = 1e-3
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    
    # Training loop from 
    # f.train_model()
    
    
    
if __name__ == '__main__':
    main()


Hello world!
